
# Dual Audience Decision Tree Analysis (HVAC/Home‑Services)
**Author:** Justin Gay  
**Date:** 2025-11-11

This single notebook includes **four sections**:
1. **Technical Analysis & Code**  
2. **Technical Stakeholder Report** (markdown)  
3. **Non‑Technical Stakeholder Report** (markdown)  
4. **Reflection** (markdown)

> **Business context**: We simulate an HVAC/home‑services business that sells yearly **maintenance plans**. Our target is whether a customer **renews** the plan for the next cycle.



## Section 1: Technical Analysis & Code
This section builds three decision trees with different `max_depth` values (3, 5, 10), evaluates accuracy (train & test), shows confusion matrices, and visualizes trees (depth 3 & 5) plus feature importance for the selected model.


In [ ]:

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, confusion_matrix

import matplotlib.pyplot as plt
from IPython.display import Markdown, display

np.random.seed(42)



### 1.1 Dataset loading and basic exploration
We **generate** a realistic synthetic dataset representative of an HVAC/home‑services company.  
**Target:** `renew_next_cycle` (1 = customer renews the annual plan; 0 = churns).


In [ ]:

# --- Generate a synthetic HVAC service plan dataset ---
n = 3000

# Demographics / home attributes
household_income = np.random.lognormal(mean=10.5, sigma=0.5, size=n)  # skewed income
home_age_years = np.random.randint(1, 80, size=n)                     # home age 1-80 years
unit_age_years = np.clip(np.round(np.random.normal(9, 4, size=n)), 0, 25)  # HVAC unit age
has_smart_thermostat = np.random.binomial(1, 0.45, size=n)
region = np.random.choice(["North", "South", "Midwest", "West"], size=n, p=[0.25, 0.35, 0.2, 0.2])

# Service / behavior attributes
contract_tenure_months = np.clip(np.round(np.random.normal(24, 10, size=n)), 1, 60)
visits_last_year = np.clip(np.round(np.random.poisson(2, size=n)), 0, 8)
emergency_calls = np.clip(np.round(np.random.poisson(0.6, size=n)), 0, 6)
satisfaction_rating = np.clip(np.round(np.random.normal(4.1, 0.8, size=n), 1), 1.0, 5.0)
price_increase_pct = np.clip(np.round(np.random.normal(7, 4, size=n), 1), 0.0, 25.0)
promo_discount_pct = np.clip(np.round(np.random.normal(5, 3, size=n), 1), 0.0, 20.0)
membership_type = np.random.choice(["Basic", "Plus", "Premium"], size=n, p=[0.5, 0.35, 0.15])
auto_pay = np.random.binomial(1, 0.55, size=n)

# Seasonality proxy: month number (1-12), some regions more likely to renew in certain months
month = np.random.randint(1, 13, size=n)

# Encode region and membership for signal
region_map = {"North": 0, "South": 1, "Midwest": 2, "West": 3}
membership_map = {"Basic": 0, "Plus": 1, "Premium": 2}

region_id = np.array([region_map[r] for r in region])
membership_id = np.array([membership_map[m] for m in membership_type])

# --- Latent propensity to renew ---
# Build a logit-like score with business realism
score = (
    0.002 * (household_income / 1000)            # higher income slightly increases renewal
    - 0.03 * (price_increase_pct)                # higher price hikes reduce renewal
    + 0.12 * satisfaction_rating                 # satisfaction is strong positive
    + 0.04 * visits_last_year                    # more visits -> stickier
    - 0.06 * emergency_calls                     # many emergencies -> friction
    + 0.03 * (contract_tenure_months / 12)       # tenure helps
    - 0.02 * unit_age_years                      # older unit can cut both ways; slightly negative here
    + 0.15 * auto_pay                            # auto-pay strongly increases renewal
    + 0.05 * has_smart_thermostat                # tech-forward customers are stickier
    + 0.06 * membership_id                       # higher tier -> more likely to renew
    + 0.02 * (region_id == 1)                    # South slightly more likely (cooling demand)
)

# Add seasonal bump (month 3-5 pre-summer tune-ups; 9-10 pre-winter tune-ups)
seasonal = np.where(np.isin(month, [3,4,5,9,10]), 0.15, 0.0)
score = score + seasonal

# Convert score to probability via logistic function
def logistic(x):
    return 1 / (1 + np.exp(-x))

p = logistic(score - 2.5)  # shift to get ~60-75% renewal rate
renew_next_cycle = np.random.binomial(1, p)

df = pd.DataFrame({
    "household_income": household_income,
    "home_age_years": home_age_years,
    "unit_age_years": unit_age_years,
    "has_smart_thermostat": has_smart_thermostat,
    "region": region,
    "region_id": region_id,
    "contract_tenure_months": contract_tenure_months,
    "visits_last_year": visits_last_year,
    "emergency_calls": emergency_calls,
    "satisfaction_rating": satisfaction_rating,
    "price_increase_pct": price_increase_pct,
    "promo_discount_pct": promo_discount_pct,
    "membership_type": membership_type,
    "membership_id": membership_id,
    "auto_pay": auto_pay,
    "month": month,
    "renew_next_cycle": renew_next_cycle
})

df.head()


In [ ]:

df.describe(include='all').transpose()



### 1.2 Train/Test Split (80/20)


In [ ]:

features = [
    "household_income", "home_age_years", "unit_age_years",
    "has_smart_thermostat", "region_id",
    "contract_tenure_months", "visits_last_year", "emergency_calls",
    "satisfaction_rating", "price_increase_pct", "promo_discount_pct",
    "membership_id", "auto_pay", "month"
]
target = "renew_next_cycle"

X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train.shape, X_test.shape



### 1.3 Three Decision Trees (max_depth = 3, 5, 10)
We train shallow, medium, and deep trees and compare performance.


In [ ]:

depths = [3, 5, 10]
results = {}

for d in depths:
    clf = DecisionTreeClassifier(max_depth=d, random_state=42)
    clf.fit(X_train, y_train)
    y_tr_pred = clf.predict(X_train)
    y_te_pred = clf.predict(X_test)

    acc_tr = accuracy_score(y_train, y_tr_pred)
    acc_te = accuracy_score(y_test, y_te_pred)
    results[d] = {
        "model": clf,
        "train_acc": acc_tr,
        "test_acc": acc_te,
        "y_tr_pred": y_tr_pred,
        "y_te_pred": y_te_pred
    }

# Display a compact summary
for d in depths:
    print(f"Depth {d}: Train Acc = {results[d]['train_acc']:.3f} | Test Acc = {results[d]['test_acc']:.3f}")



### 1.4 Confusion Matrices


In [ ]:

for d in depths:
    cm = confusion_matrix(y_test, results[d]["y_te_pred"], labels=[0,1])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0,1])
    plt.figure(figsize=(4,4))
    disp.plot(values_format='d')
    plt.title(f"Confusion Matrix (Test) – Depth {d}")
    plt.show()



### 1.5 Tree Visualizations
For readability, we plot the **shallow** (depth 3) and **medium** (depth 5) trees.


In [ ]:

for d in [3, 5]:
    clf = results[d]["model"]
    plt.figure(figsize=(18, 8))
    plot_tree(
        clf,
        feature_names=features,
        class_names=["no_renew", "renew"],
        filled=False,  # avoid color to keep visuals simple/portable
        rounded=True,
        max_depth=d
    )
    plt.title(f"Decision Tree (max_depth={d})")
    plt.show()



### 1.6 Feature Importances (Chosen Model)
We select the model with the **best test accuracy** (ties break in favor of the **simpler** tree).


In [ ]:

# Pick best by test accuracy; break ties by smaller depth
best_depth = sorted(depths, key=lambda d: (-results[d]["test_acc"], d))[0]
best_model = results[best_depth]["model"]
best_test_acc = results[best_depth]["test_acc"]
best_train_acc = results[best_depth]["train_acc"]

importances = best_model.feature_importances_
imp_df = pd.DataFrame({"feature": features, "importance": importances}).sort_values("importance", ascending=False)

print("Selected depth:", best_depth)
print(f"Train Acc: {best_train_acc:.3f} | Test Acc: {best_test_acc:.3f}")
imp_df.head(10)


In [ ]:

plt.figure(figsize=(8,5))
plt.barh(imp_df["feature"], imp_df["importance"])
plt.gca().invert_yaxis()
plt.title(f"Feature Importance – Chosen Tree (depth={best_depth})")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.show()



## Section 2: Technical Stakeholder Report
*(Markdown generated programmatically using results from Section 1)*


In [ ]:

def technical_report():
    lines = []

    lines.append("### Dataset & Prediction Goal")
    lines.append("We model **renewal** of an HVAC annual maintenance plan (`renew_next_cycle`). Predicting renewal helps forecast recurring revenue and plan outbound retention efforts.")

    lines.append("\n### Methodology Summary")
    lines.append("- 80/20 **train/test split** with `random_state=42`, stratified on the target.")
    lines.append("- Trained **DecisionTreeClassifier** at three depths: **3**, **5**, **10**.")
    lines.append("- Evaluated **accuracy** on training and test sets; plotted confusion matrices; visualized trees (depth 3 & 5).")

    # Performance table
    lines.append("\n### Model Performance Comparison")
    for d in [3,5,10]:
        tr = results[d]["train_acc"]
        te = results[d]["test_acc"]
        lines.append(f"- Depth {d}: **Train**={tr:.3f}, **Test**={te:.3f}")

    lines.append("\n### Overfitting Analysis")
    lines.append("We compare train vs. test accuracy to diagnose **overfitting**.")
    # Identify overfitting pattern
    overfit_notes = []
    for d in [3,5,10]:
        gap = results[d]["train_acc"] - results[d]["test_acc"]
        if gap > 0.04:
            overfit_notes.append(f"Depth {d} shows a notable generalization gap (~{gap:.2f}).")
    if overfit_notes:
        lines.extend([f"- {note}" for note in overfit_notes])
    else:
        lines.append("- No large gaps detected; deeper trees may still be brittle on new data.")

    lines.append("\n### Tree Structure Comparison")
    lines.append("The **depth‑3** tree is compact and uses a handful of high‑signal splits (often satisfaction, price increase, tenure/auto‑pay).")
    lines.append("The **depth‑5** tree captures additional interactions (e.g., seasonality via `month`, tier via `membership_id`) with manageable complexity.")
    lines.append("The **depth‑10** tree is much larger, more tailored to idiosyncrasies in the training set, and at higher risk of variance.")

    lines.append("\n### Technical Recommendation")
    lines.append(f"We selected **depth {best_depth}** with **Test Acc = {best_test_acc:.3f}** as the operating point. ")
    lines.append("It best balances the **bias‑variance tradeoff** for this dataset. "
                 "If results between depth 3 and 5 are close, prefer the shallower tree for stability and interpretability.")

    display(Markdown("\n".join(lines)))

technical_report()



## Section 3: Non‑Technical Stakeholder Report
*(Plain‑language summary for managers/executives; no ML jargon)*


In [ ]:

def non_technical_report():
    top3 = imp_df.head(3)["feature"].tolist()
    overall = best_test_acc

    # Translate top features into plain English
    pretty = {
        "satisfaction_rating": "customer satisfaction score",
        "price_increase_pct": "size of the price increase",
        "contract_tenure_months": "how long they’ve been with us",
        "auto_pay": "whether they use auto‑pay",
        "membership_id": "membership tier (Basic/Plus/Premium)",
        "visits_last_year": "number of technician visits last year",
        "emergency_calls": "number of emergency calls",
        "has_smart_thermostat": "whether they use a smart thermostat",
        "unit_age_years": "age of the HVAC unit",
        "household_income": "estimated household income",
        "home_age_years": "age of the home",
        "promo_discount_pct": "size of the promo discount",
        "region_id": "customer region",
        "month": "time of year"
    }

    def name_map(f):
        return pretty.get(f, f)

    lines = []
    lines.append("### Business Question")
    lines.append("Which **customers are most likely to renew** their yearly maintenance plan so we can plan staffing and prioritize outreach?")

    lines.append("\n### Key Findings")
    lines.append(f"Top factors driving renewal in this analysis include: **{name_map(top3[0])}**, **{name_map(top3[1])}**, and **{name_map(top3[2])}**.")
    lines.append("In plain terms, happier customers, smaller price hikes, and longer‑tenured/auto‑pay customers are more likely to renew.")

    lines.append("\n### Performance Summary")
    lines.append(f"Our analysis made **correct predictions** about renewal **{overall*100:.1f}%** of the time on new (unseen) data.")

    lines.append("\n### Real‑World Example")
    lines.append("Think of a flowchart that asks simple yes/no or threshold questions.")
    lines.append("For instance: if a customer’s satisfaction is high and their price increase is small, the analysis tends to predict **renewal**;")
    lines.append("if satisfaction is low and the price hike is large, it tends to predict **non‑renewal**.")

    lines.append("\n### Business Recommendations")
    lines.append("1) **Protect satisfaction**: route at‑risk customers (low score) to a save‑desk for proactive follow‑up.")
    lines.append("2) **Targeted pricing**: avoid large price hikes for long‑tenured customers; test smaller increases paired with value messages.")
    lines.append("3) **Promote auto‑pay and higher tiers**: nudge customers to auto‑pay/Premium with small incentives to increase stickiness.")

    lines.append("\n### Limitations")
    lines.append("This analysis is based on historical patterns and can miss sudden changes (e.g., economic shocks, competitor promotions).")
    lines.append("Customer behavior can shift over seasons; predictions are best used as **guidance**, not guarantees.")

    display(Markdown("\n".join(lines)))

non_technical_report()



## Section 4: Reflection
Translating technical work into plain language is challenging because the most **useful** details (hyperparameters, bias‑variance tradeoff, overfitting signals) are exactly the terms a non‑technical audience doesn’t use day‑to‑day. The risk is either **oversimplifying** (and losing nuance) or **overloading** (and losing the audience).

Two strategies helped here:
1. **Keep the structure identical** across audiences but **change the vocabulary**—e.g., “correct predictions” instead of “accuracy,” “factors” instead of “features,” and “flowchart” instead of “decision tree.”  
2. **Anchor every claim to a concrete business action** (save‑desk for low‑satisfaction customers, targeted price changes for long‑tenured accounts, nudges to auto‑pay). This maintains credibility without forcing the audience through ML terminology.
